# Machine Learning

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Victor Uc Cetina <[victoruccetina@gmail.com](mailto:victoruccetina@gmail.com)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

In [ ]:
import time
import matplotlib.pyplot as plt
from sklearn import datasets
from tensorflow import keras

In [ ]:
def load_iris():
    _dataset_iris = datasets.load_iris()
    _x = _dataset_iris.data
    _iris_target = _dataset_iris.target
    _y = keras.utils.to_categorical(_iris_target, dtype='uint8')
    return _x, _y


x, y = load_iris()
x.shape, y.shape

In [ ]:
weights_list = []

In [ ]:
class WeightsCollectorCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global weights_list
        i_weights = self.model.get_weights()
        weights_list.append(i_weights)

In [ ]:
def plot_weights(_activation: str, optimizer_name: str):
    global weights_list
    rr = range(len(weights_list))
    _layers_count = len(weights_list[0]) // 2
    plt.title('"Weights" L:{} A:{} O:{}'.format(_layers_count, _activation, optimizer_name))
    for l in range(_layers_count):  # layers
        l *= 2
        b = l + 1
        _in, _out = weights_list[0][l].shape
        for _i in range(_in):  # inputs
            for _o in range(_out):  # outputs
                plt.plot([weights_list[i][l][_i][_o] for i in rr])
                plt.plot([weights_list[i][b][_o] for i in rr])
    plt.savefig(f'img/weights_l_{_layers_count}_a_{_activation}_o_{optimizer_name}.png'.lower(), dpi=200)
    plt.show()

In [ ]:
def experiment(_layers_count: int, _activation: str, _optimizer):
    global x, y, weights_list
    weights_list = []
    input_shape = x.shape[1:]  # (4,)
    output_neurons = y.shape[1]  # 3
    model_layers = [keras.layers.Dense(2, activation=_activation, name='layer_0', input_shape=input_shape)]
    for i in range(1, _layers_count - 1):
        model_layers.append(keras.layers.Dense(2, activation=_activation, name=f'layer_{i}'))
    model_layers.append(keras.layers.Dense(output_neurons, activation='softmax', name='layer_out'))
    model = keras.Sequential(model_layers)
    model.summary()
    print(len(model_layers))
    model.compile(
        optimizer=_optimizer,
        loss=keras.losses.MeanSquaredError()
    )
    model.fit(x, y, epochs=100, callbacks=[WeightsCollectorCallback()])
    plot_weights(_activation, _optimizer.name)

In [ ]:
# > * Número de capas: {2,3,4,5,6}
# > * Función de activación: {Sigmoid, ReLu, Tanh}
# > * Optimizer: {SGD, Adam}

e_layers_count = [2, 3, 4, 5, 6]
e_activations = ['sigmoid', 'relu', 'tanh']

for lc in e_layers_count:
    for a in e_activations:
        time_start = time.time()
        experiment(lc, a, keras.optimizers.SGD(learning_rate=0.03))
        experiment(lc, a, keras.optimizers.Adam(learning_rate=0.03))
        print('\033[92m')
        print(f'Experimento con {lc} capas, {a} y SGD: {time.time() - time_start} segundos')
        print('\033[0m')